<a href="https://colab.research.google.com/github/mohripan/Kaggle-Training/blob/main/Bangsat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
!wget --no-check-certificate \
  https://dicodingacademy.blob.core.windows.net/picodiploma/ml_pemula_academy/rockpaperscissors.zip \
  -O /tmp/rockpaperscissors.zip

--2021-07-03 11:35:26--  https://dicodingacademy.blob.core.windows.net/picodiploma/ml_pemula_academy/rockpaperscissors.zip
Resolving dicodingacademy.blob.core.windows.net (dicodingacademy.blob.core.windows.net)... 52.239.197.36
Connecting to dicodingacademy.blob.core.windows.net (dicodingacademy.blob.core.windows.net)|52.239.197.36|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 322873683 (308M) [application/zip]
Saving to: ‘/tmp/rockpaperscissors.zip’

/tmp/rockpapersciss 100%[===================>] 307.92M  5.25MB/s    in 49s     

2021-07-03 11:36:16 (6.34 MB/s) - ‘/tmp/rockpaperscissors.zip’ saved [322873683/322873683]



In [3]:
import zipfile, os

In [4]:
loc_zip = '/tmp/rockpaperscissors.zip'
zip_ref = zipfile.ZipFile(loc_zip, 'r')
zip_ref.extractall('/tmp')
zip_ref.close()

base_dir = '/tmp/rockpaperscissors/rps-cv-images'

In [5]:
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, GlobalAveragePooling2D, Conv2D, MaxPooling2D,Activation,Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

In [6]:
train_datagen = ImageDataGenerator( rescale = 1.0/255,
                                          rotation_range=20,
                                          width_shift_range=0.2,
                                          height_shift_range=0.2,
                                          shear_range=0.2,
                                          zoom_range=0.2,
                                          horizontal_flip=True,
                                          fill_mode='nearest',
                                           validation_split=0.4
                                  )


train_generator = train_datagen.flow_from_directory(base_dir,
                                                    batch_size=32,
                                                    class_mode='categorical',
                                                    target_size=(80, 60),
                                                   subset='training')

validation_datagen = ImageDataGenerator(rescale = 1.0/255,
                                        validation_split=0.4)


validation_generator = validation_datagen.flow_from_directory(base_dir,
                                                    batch_size=32,
                                                    class_mode='categorical',
                                                    target_size=(80, 60),
                                                    subset='validation')

Found 1314 images belonging to 3 classes.
Found 874 images belonging to 3 classes.


In [7]:
model = Sequential()
model.add(Conv2D(32, (3,3), input_shape=(80, 60, 3), activation='relu'))
model.add(MaxPooling2D(2, 2))
model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(MaxPooling2D(2, 2))
model.add(Flatten())
model.add(Dense(units=512, activation='relu'))
model.add(Dense(units=256, activation='relu'))
model.add(Dense(units=3, activation='softmax'))

model.summary()

model.compile(loss = 'categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 78, 58, 32)        896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 39, 29, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 37, 27, 32)        9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 18, 13, 32)        0         
_________________________________________________________________
flatten (Flatten)            (None, 7488)              0         
_________________________________________________________________
dense (Dense)                (None, 512)               3834368   
_________________________________________________________________
dense_1 (Dense)              (None, 256)               1

In [8]:
from tensorflow.keras.callbacks import ReduceLROnPlateau
learning_rate_reduction = ReduceLROnPlateau(monitor='val_acc',
                                            patience=2,
                                            verbose=1,
                                            factor=0.5,
                                            min_lr=0.000001)

In [9]:
his=model.fit(train_generator,validation_data=validation_generator, epochs = 30,
                    verbose = 1, callbacks=[learning_rate_reduction])

Epoch 1/30
42/42 [==============================] - 9s 154ms/step - loss: 1.0703 - accuracy: 0.4132 - val_loss: 0.8914 - val_accuracy: 0.7346
Epoch 2/30
42/42 [==============================] - 6s 146ms/step - loss: 0.9005 - accuracy: 0.5875 - val_loss: 0.6278 - val_accuracy: 0.7780
Epoch 3/30
42/42 [==============================] - 6s 148ms/step - loss: 0.7683 - accuracy: 0.6773 - val_loss: 0.3915 - val_accuracy: 0.9233
Epoch 4/30
42/42 [==============================] - 6s 146ms/step - loss: 0.4458 - accuracy: 0.8318 - val_loss: 0.1549 - val_accuracy: 0.9645
Epoch 5/30
42/42 [==============================] - 6s 145ms/step - loss: 0.3406 - accuracy: 0.8798 - val_loss: 0.1153 - val_accuracy: 0.9634
Epoch 6/30
42/42 [==============================] - 6s 145ms/step - loss: 0.2930 - accuracy: 0.8957 - val_loss: 0.1048 - val_accuracy: 0.9737
Epoch 7/30
42/42 [==============================] - 6s 144ms/step - loss: 0.2122 - accuracy: 0.9262 - val_loss: 0.0950 - val_accuracy: 0.9737
Epoch 